<a name="top"></a>
<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://cdn.miami.edu/_assets-common/images/system/um-logo-gray-bg.png" alt="Miami Logo" style="height: 98px;">
</div>

<div style="float:right; width:98 px; height:98px;">
<img src="https://media.licdn.com/dms/image/C4E0BAQFlOZSAJABP4w/company-logo_200_200/0/1548285168598?e=2147483647&v=beta&t=g4jl8rEhB7HLJuNZhU6OkJWHW4cul_y9Kj_aoD7p0_Y" alt="STI Logo" style="height: 98px;">
</div>


<h1>Data Download: UFS-S2S</h1>
By: Kayla Besong, PhD
    <br>
Last Edited: 01/09/24
<br>
<br>    
<br>
This code is designed to download fire weather variables for the UFS S2S from AWS. A complementary notebook 'Data_Grab_Functions.ipynb' hosts the suite of functions tailored and necessary to run this script. This notebook was designed with intentention for inputs such as start and end date, variables to be changed depending on user need. Below it is tailored to fire weather metrics, if additional or changed variables are desired it is often necessary to find the level or surface your variable is on and align it properly with the input dictionary. Each function will generate a file tree and save the files to that file tree structure, with the only object returned being a list of unavailable files for your specified dates/hours/variables. Lastly, be patient.
    
<div style="clear:both"></div>
</div>

<hr style="height:2px;">

### The integral notebook of functions to run

In [1]:
%run ../../Universal_Functions/Data_Grab_Functions.ipynb

# UFS S2S

source: AWS
<br>
file type: grib2
<br>
dates available from: 2011-04-01 to Present initiated on the 1st and 15th of every month
<br>
analysis time steps: 0 to forecast hr 840 by 6 
<br>
domain: the domain is subset to 85N-0N, 180W-360W in this code to cover all of North America including Alaska and Hawai'i; global available 
<br>
<br>
Resources on UFS S2S:
<br>
https://registry.opendata.aws/noaa-ufs-s2s/
<br>
https://noaa-ufs-prototypes-pds.s3.amazonaws.com/index.html
<br>
https://vlab.noaa.gov/web/ufs-r2o/mrw-s2s
<br>
<br>
Estimated time to run: 1-year, 4x daily, 11 vars = ~XXXXXX hours
<br>
<br>
output:
<br>
1. A list of all missing or incomplete files
2. A file tree structured UFS_S2S/prototypenum/variable/ufs_prototypenum_variable_initYEARMONDAY_leadtime.nc
<br>
<br>
NOTE: to read grib2 off of AWS, this code will create an additional directory 'temp_ufs_files/files' to store temporary files that can manually be deleted later. Controlling where the files are cached has proven difficult. This method may work on your machine, it may not depending on what is set up. Within the ufs_s2s_grabber function in the Data_Grab_Functions.ipynb there are 5 lines commented out that delete the cached grib files as you go. If you find that all the cached files are causing a problem, you can uncomment these lines to automatically delete them. It is advised to print 'file' before doing so to become aware of where the cached files are being stored or generated first. This method uses shutil.rmtree() and could delete things you do not want deleted. 

In [ ]:
### Variables are set up in a dictionary format as not all variables are stored with the same coordinates.
### The keys represent basically whatever you want. Each variable of interest is stored in a list of lists as the 'values' of the dictionary.
### The additional list is the 'filter_by_keys' input for each of the variables required by xarray-cfgrib to obtain the variables of interest.
### Each variable at a minimum requires 'typeOfLevel' while others require more such as below. If you add/change variables, some investigation
### as to which filters are needed may be required. Presently, you cannot provide multiple choices per key-value pair. For example, for the first
### two they both are on the typeOfLevel: 'heightAboveGround' but require seperate 'level' values...level: [2, 10] will return an empty xr.dataset
### due to xarray-cfgrib not "liking" the different 'coordinates'. Best of luck! 

variables = {'heightAboveGround2': [['t2m', 'r2'], {'typeOfLevel': 'heightAboveGround', 'level': 2}],
             'heightAboveGround10': [['u10', 'v10'], {'typeOfLevel': 'heightAboveGround', 'level': 10}],
             'surface': [['gust', 'hpbl', 'cape', 'prate', 'lsm', 'hindex'], {'typeOfLevel': 'surface', 'stepType': 'instant'}],
             'depthBelowLandLayer': [['soilw'], {'typeOfLevel': 'depthBelowLandLayer'}]}
             

In [ ]:
### inputs to the hrrr_grabber function are established here. 
### start_date, end_date, and output_dir = strings
### to_lead_time = integer --> is to the lead time you want from 0 to 840 by 6
### domain == array of integers --> [N, S, E, W] presently set up for NA domain

start_date = '2014-03-15'
end_date = '2018-04-01'
to_lead_time = 840
output_dir = 'UFS_S2S'
domain = [85, 0, 180, 360]


In [ ]:
%%time

### Calling the UFS grabber function to download all data of interest. The only item that is returned is a 
### list of files that does not exists (is missing or otherwise) for one to be aware of and double check on. 

non_exist_ufs = ufs_s2s_grabber(start_date, end_date, to_lead_time, variables, output_dir, domain, 6, '/raid60B/s2sfire/NOAA_S2S/temp_ufs_files/')